In [1]:
!pip install transformers datasets pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import random

In [3]:
papers_dataset = load_dataset("scientific_papers","arxiv")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for scientific_papers contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/scientific_papers
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warn

Generating train split:   0%|          | 0/203037 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6436 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6440 [00:00<?, ? examples/s]

In [4]:
feature_to_remove = "section_names"

for split, dataset in papers_dataset.items():
    papers_dataset[split] = dataset.remove_columns(feature_to_remove)

In [5]:
# Removing Articles with blank text from train dataset
i = 0
indices_to_remove = []
while i < len(papers_dataset["train"]):
  if papers_dataset["train"][i]["article"] == "" or papers_dataset["train"][i]["abstract"] == "":
    indices_to_remove.append(i)
  i += 1

filtered_dataset = papers_dataset["train"].filter(lambda example, idx: idx not in indices_to_remove, with_indices=True)
papers_dataset["train"] = filtered_dataset

i = 0
indices_to_remove = []
while i < len(papers_dataset["validation"]):
  if papers_dataset["validation"][i]["article"] == "" or papers_dataset["validation"][i]["abstract"] == "":
    indices_to_remove.append(i)
  i += 1

filtered_dataset = papers_dataset["validation"].filter(lambda example, idx: idx not in indices_to_remove, with_indices=True)
papers_dataset["validation"] = filtered_dataset

print("Records with blank data removed")

Filter:   0%|          | 0/203037 [00:00<?, ? examples/s]

Filter:   0%|          | 0/6436 [00:00<?, ? examples/s]

Records with blank data removed


In [6]:
papers_dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract'],
        num_rows: 202914
    })
    validation: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6436
    })
    test: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6440
    })
})

In [10]:
train_data = papers_dataset["train"]

# Number of parts to divide the dataset into
num_parts = 2000

# Calculate the number of samples in each part
num_samples_per_part = len(train_data) // num_parts

# Shuffle the dataset before splitting
train_data = train_data.shuffle()

# Divide the dataset into parts
data_parts = [train_data.shard(num_parts, i) for i in range(num_parts)]

# Store each part locally as a CSV file
for i, part in enumerate(data_parts):
    part_df = pd.DataFrame(part)
    part_df.to_csv(f"/content/drive/MyDrive/Submission/BE Project Group No 31/Data/training_data{i+1}.csv", index=False,escapechar="\\")

In [11]:
val_data = papers_dataset["validation"]

# Number of parts to divide the dataset into
num_parts = 2000

# Calculate the number of samples in each part
num_samples_per_part = len(val_data) // num_parts

# Shuffle the dataset before splitting
val_data = val_data.shuffle()

# Divide the dataset into parts
data_parts = [val_data.shard(num_parts, i) for i in range(num_parts)]

# Store each part locally as a CSV file
for i, part in enumerate(data_parts):
    part_df = pd.DataFrame(part)
    part_df.to_csv(f"./drive/MyDrive/Submission/BE Project Group No 31/Data/val_data{i+1}.csv", index=False,escapechar="\\")

In [12]:
test_data = papers_dataset["test"]

# Number of parts to divide the dataset into
num_parts = 120

# Calculate the number of samples in each part
num_samples_per_part = len(test_data) // num_parts

# Shuffle the dataset before splitting
test_data = test_data.shuffle()

# Divide the dataset into parts
data_parts = [test_data.shard(num_parts, i) for i in range(num_parts)]

# Store each part locally as a CSV file
for i, part in enumerate(data_parts):
    part_df = pd.DataFrame(part)
    part_df.to_csv(f"./drive/MyDrive/Submission/BE Project Group No 31/Data/test_data{i+1}.csv", index=False,escapechar="\\")
